# Versions
1. Version 3 - Used pretrain model weights with glove 6b 
2. Version 4 - Used pretrain model weights with glove 840d 400d

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/clr-glove-weights/__notebook_source__.ipynb
/kaggle/input/clr-glove-weights/submission.csv
/kaggle/input/clr-glove-weights/golve_840b300d.h5
/kaggle/input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl
/kaggle/input/commonlitreadabilityprize/sample_submission.csv
/kaggle/input/commonlitreadabilityprize/train.csv
/kaggle/input/commonlitreadabilityprize/test.csv


In [2]:
import pandas as pd 
import numpy as np 
from tqdm import tqdm
import re

from sklearn.model_selection import train_test_split,KFold
from sklearn.metrics import mean_squared_error

import nltk
from nltk.corpus import stopwords

import tensorflow as tf
from tensorflow.keras.layers import LSTM,RNN,Conv2D,Dense,Flatten,GlobalAveragePooling2D,Embedding,Bidirectional,Input,Dropout,Conv1D,MaxPooling1D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam,SGD


SEED=42

In [3]:
paths=["/kaggle/input/commonlitreadabilityprize/sample_submission.csv",
       '/kaggle/input/commonlitreadabilityprize/train.csv',
       '/kaggle/input/commonlitreadabilityprize/test.csv']
df_train=pd.read_csv(paths[1])
df_test=pd.read_csv(paths[2])
df_ss=pd.read_csv(paths[0])

In [4]:
def clean(string):
    pattern='\n'
    pattern1= '\''
    test= re.sub(pattern,'',string)
    string= re.sub(pattern1,'',string)

    return string

In [5]:
#remove stopwords 
nltk.download('stopwords')
stop=stopwords.words('english')


def remove_stopwords(df):
    new_text=[]
    for i in range(len(df)):
        test=[j for j in df['excerpt'][i].split() if j not in stop]
        new_text.append(' '.join(test))
    new_text=pd.Series(new_text,name='cleaned_text')
    df=pd.concat([df,new_text],axis='columns',copy=False)
    return df

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [6]:
df_train['excerpt']=df_train['excerpt'].map(clean)
df_test['excerpt']=df_test['excerpt'].map(clean)


In [7]:
df_train= remove_stopwords(df_train)
df_test= remove_stopwords(df_test)


In [8]:
length=[]
for i in df_train['cleaned_text']:
    length.append(len(i))
    
length= np.array(length)
print(length.mean(),length.min(),length.max())

681.4446012702894 391 1101


In [9]:
X=df_train['cleaned_text']
y=df_train['target']
test=df_test['cleaned_text']



In [10]:
VOCAB= 25000
max_len=681
oov_token='<OOV_TOKEN>'
truncate_type='post'
padding_type='post'
embedding_dim=16


In [11]:
#tokenize
tokenizer=Tokenizer(oov_token=oov_token,num_words=VOCAB)
tokenizer.fit_on_texts(X)
word_index = tokenizer.word_index
print(len(word_index))

#

train_sequences= tokenizer.texts_to_sequences(X)
test_sequences= tokenizer.texts_to_sequences(test)

train_padding = pad_sequences(train_sequences, maxlen=max_len, padding= padding_type, truncating= truncate_type)
test_padding = pad_sequences(test_sequences, maxlen=max_len, padding= padding_type , truncating= truncate_type)

28177


In [12]:
import pickle

with open('../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl', 'rb') as fp:
    embeddings_index  = pickle.load(fp)
    
# create an embedding matrix for the words we have in the dataset
embedding_matrix = np.zeros((len(word_index) + 1, 300))
for word, i in tqdm(word_index.items()):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
        
print(f'Shape of Embedding: {embedding_matrix.shape}')

100%|██████████| 28177/28177 [00:00<00:00, 215223.93it/s]

Shape of Embedding: (28178, 300)


In [13]:
input_layer = Input(shape=(681,))
x= embedding_layer= Embedding(len(word_index)+1,300,trainable=False,weights=[embedding_matrix])(input_layer)
x= Conv1D(32,3,activation='relu')(x)
x=MaxPooling1D(pool_size=2)(x)
x= Bidirectional(LSTM(150))(x)
x=Dense(128,activation='relu')(x)
x=Dense(64,activation='relu')(x)
x=Dense(32,activation='relu')(x)
x=Dense(16,activation='relu')(x)
predictions = Dense(1,activation='linear')(x)

model1=Model(inputs=input_layer, outputs= predictions)

print(model1.summary())


model1.compile(
    optimizer= Adam(learning_rate=1e-5),
    loss='mse',
    metrics='mae'
)

model_checkpoint=ModelCheckpoint('golve_200.h5',monitor='loss',save_best_only=True,mode='min')
early_stopping=EarlyStopping(monitor="loss",min_delta=0,patience=10,verbose=0,mode="min",restore_best_weights=True)
reduce_lr=ReduceLROnPlateau(monitor="loss",factor=0.2,patience=10,min_lr=0.00001)


Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 681)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 681, 300)          8453400   
_________________________________________________________________
conv1d (Conv1D)              (None, 679, 32)           28832     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 339, 32)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 300)               219600    
_________________________________________________________________
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256  

In [14]:
model1.load_weights('../input/clr-glove-weights/golve_840b300d.h5')

In [15]:
y_pred = model1.predict(test_padding)

In [16]:
sub_scores=[]
for i in y_pred:
    sub_scores.append(i[0])


In [17]:
sub=pd.DataFrame({'id':df_ss['id'],'target':sub_scores})
sub.to_csv('submission.csv',index=False)
sub.head()

,id,target
0,c0f722661,-1.280372
1,f0953f0a5,-0.258921
2,0df072751,-0.744956
3,04caf4e0c,-2.258774
4,0e63f8bea,-1.616462
